# DETECTING PENTAS AND QUADRAS

In [47]:
from PIL import Image
import cv2
from IPython.display import clear_output
import pytesseract
from pytube import YouTube
import os
import datetime

In [60]:
def crop_image(x1, x2, y1, y2):
    """crops the image for detecting the text better and returns"""
    
    imgage = Image.open("frame.png")
    area = (x1, x2, y1, y2)
    cropped_image = imgage.crop(area)
    return cropped_image


def get_timestamp(video_capture, fps):
    """returns exact time of the current frame"""
    
    timestamps = [video_capture.get(cv2.CAP_PROP_POS_MSEC)]
    calculate_timestamps = [0.0]
    timestamps.append(video_capture.get(cv2.CAP_PROP_POS_MSEC))
    calculate_timestamps.append(calculate_timestamps[-1] + 1000/fps)
    time, _ = zip(timestamps, calculate_timestamps)
    time = time[0] // 1000
    return time


def detect_pentas_quadras(video_path, player, video_name, video_counter, videos_count):
    """detects pentas and quadras in videos and returns as a list"""
    
    video_capture = cv2.VideoCapture(video_path)
    #fps = vidcap.get(cv2.cv.CV_CAP_PROP_FPS)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    total_seccond = frame_count // fps
    pentas = []
    quadras = []
    seccond_counter = 0
    #width = vcap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)   # float
    #width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
    #height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    while(video_capture.isOpened()):
        for i in range(fps):
            frame_exists, image = video_capture.read()
        seccond_counter += 1
        
        #cv2.imwrite("frame.jpg", image)
        #img = cv2.imread('im.png')
        #Pıl image.save("name")
        
        if frame_exists:
            cv2.imwrite("frame.png", image)
        else:
            break

        cropped_image_penta = crop_image(400, 0, 900, 250)
        result_penta = pytesseract.image_to_string(cropped_image_penta, lang="eng")
        
        cropped_image_quadra = crop_image(450, 50, 800, 175)
        result_quadra = pytesseract.image_to_string(cropped_image_quadra, lang="tur")

        if ("BESTE" in result_penta) or ("BES" in result_penta) or\
                                    ("BEŞTE" in result_quadra)or ("BEŞ" in result_quadra):
            time = get_timestamp(video_capture, fps)
            pentas.append([time, player, video_name])
        elif ("DÖRTTE" in result_quadra) or ("DÖRT" in result_quadra) or\
                                    ("DORT" in result_penta) or ("DORTTE" in result_penta):
            time = get_timestamp(video_capture, fps)
            quadras.append([time, player, video_name])
        
        clear_output(wait=True)
        print(seccond_counter, "/", total_seccond, " | ", video_counter, "/", videos_count)
    
    video_capture.release()
    return pentas, quadras


def download_videos():
    """downloads the videos"""
    
    with open("videos.txt", "r") as file:
        videos = file.read().split("***")

    video_count = 0
    video_links = []
    for video in videos:
        video_ = video.split("\n")
        player = video_[0]
        links = video_[1:]
        video_links.append((player, links))
        video_count += len(links)

    download_counter = 0
    for name, links in video_links:
        link_count = len(links)
        for link in links:
            yt = YouTube(link).streams.first().download('videos/' + name)
            clear_output(wait=True)
            download_counter += 1
            print(download_counter, "/", video_count)


def get_videos_list(players):
    """returns list of videos of a players"""
    
    all_videos = []
    for player in players:
        for r, d, f in os.walk("videos/" + yayinci):
            for file in f:
                if '.mp4' in file:
                    all_videos.append(os.path.join(r, file))
    return all_videos


def get_quadras_pentas():
    """returns pentas and quadras in a list"""
    
    video_counter = 0
    count_all_videos = len(all_videos)
    pentas = []
    quadras = []
    
    for video in all_videos:
        splits = video.split("/")
        player = splits[1]
        video_name = splits[2][:50]
        video_counter += 1
        penta, quadra = detect_pentas_quadras(video, yayinci, video_name, video_counter, count_all_videos)
        pentas.append(penta)
        quadras.append(quadra)
    return pentas, quadras


def print_and_save_times(penta_list, quadra_list):
    """saves the information of quadras and pentas info a file"""
    
    print("Pentas", "********")
    to_save = "Pentas\n"
    for pentas in penta_list:
        for record in pentas:
            record[0] = str(int(record[0]) // 60) + ":" + str(int(record[0]%60))
            print(record)
            to_save += ', '.join(record) + "\n"
    
    print("Quadras", "********")
    to_save += "\nQuadras\n"
    for quadras in quadra_list:
        for record in quadras:
            record[0] = str(int(record[0]) // 60) + ":" + str(int(record[0]%60))
            print(record)
            to_save += ', '.join(record) + "\n"
    
    add_file_name = str(datetime.datetime.now())[5:16]
    with open("pentas_quadras_" + add_file_name +".txt", "w") as file:
        file.write(to_save)

In [53]:
download_videos()

3 / 3


In [57]:
get_videos_list(["baris",])
pentas, quadras = get_quadras_pentas()

659 / 659  |  3 / 3


In [61]:
print_and_save_times(pentas, quadras)

Pentas ********
['19:44', 'baris', 'PİNGİM VAR DEDİ OYUNDAN ÇIKTI !! PENTA ATTIM !! 4V']
['9:56', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
['9:57', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
Quadras ********
['0:19', 'baris', 'PİNGİM VAR DEDİ OYUNDAN ÇIKTI !! PENTA ATTIM !! 4V']
['0:21', 'baris', 'PİNGİM VAR DEDİ OYUNDAN ÇIKTI !! PENTA ATTIM !! 4V']
['19:39', 'baris', 'PİNGİM VAR DEDİ OYUNDAN ÇIKTI !! PENTA ATTIM !! 4V']
['19:40', 'baris', 'PİNGİM VAR DEDİ OYUNDAN ÇIKTI !! PENTA ATTIM !! 4V']
['19:41', 'baris', 'PİNGİM VAR DEDİ OYUNDAN ÇIKTI !! PENTA ATTIM !! 4V']
['4:47', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
['7:38', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
['7:39', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
['9:53', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
['9:54', 'baris', '1V4 İMKANSIZ QUADRAYI ATMAK !! ZEDİ 90LARA GÖNDERD']
['9:55', 'baris', '1V4 İMKA